In [ ]:
# CHECK OUR DEPENDENCIES THAT WE HAVE 
!pip list

In [ ]:
from dotenv import load_dotenv
import os 

load_dotenv()

print(os.environ.get('RIOT_API_KEY'))

In [ ]:
import pygsheets

service_account = pygsheets.authorize(service_account_file='../JSONS/spreadsheet-automater-442917-df4c20260fbc.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1BNii3rkiKVw5BwEYmTTuqTI1V9ce46t52pXRCX_G7TM/edit?usp=sharing')

In [ ]:

test = sheet.worksheet('title','TEST')
print(test)

In [ ]:
test_df = test.get_as_df()

test_df['random_numbers'] = 1

test_df



In [ ]:
# Writing data into google sheets programatically
test.set_dataframe(test_df,'A1',copy_head=True)

In [ ]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

# GET PUUID for GIVEN USER AND TAG LINE 
RIOT_API_KEY = os.environ.get('RIOT_API_KEY')
USERNAME = 'Adragegood'   
TAG_LINE = 'NA1'
PUUID = 'M8xApHTCkty4N-Jz7ZVP5j5p0CVnZKM8ajEV6bn6ezq3Xgo9AG1mFPlvhlaWRzp6Vy47vTlR8GGuRw'
SUM_ID = 'TgyaCz98lKsKQF83Uo0hNRArNY7kH9lT8KvBQusfMaj9fuU'
REGION = 'americas'

def get_puuid(gameName=None,tagLine=None,api_key=None,region=None):
    link = f'https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'
    response = requests.get(link)
    return response.json()["puuid"]

def get_name_and_tag(puuid=None,api_key=None,region=None):
    link = f'https://{region}.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'
    response = requests.get(link)
    id = {
        'gameName' : response.json()['gameName'],
        'tagLine' : response.json()['tagLine']
    }
    return id





In [ ]:
# TESTING GET PUUID AND AS WELL AS GET NAME AND TAG LINE WITH A PUUID
get_puuid(USERNAME,TAG_LINE,RIOT_API_KEY,REGION)



In [ ]:
get_name_and_tag(PUUID,RIOT_API_KEY,REGION)

In [ ]:
# LEAGUE - V4 is ranked latter for riot API

import pandas as pd 


def get_ladder(top=None):
    root = 'https://na1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    grand_m = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'


    chall_response = requests.get(root + chall + '?api_key=' + RIOT_API_KEY)
    gm_response = requests.get(root + grand_m + '?api_key=' + RIOT_API_KEY)
    m_response = requests.get(root + master + '?api_key=' + RIOT_API_KEY)

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints',ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints',ascending=False).reset_index(drop=True)
    m_df = pd.DataFrame(m_response.json()['entries']).sort_values('leaguePoints',ascending=False).reset_index(drop=True)


    ladder = pd.concat([chall_df,gm_df,m_df]).reset_index(drop=True)[:top]
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] +=1

    return ladder


In [ ]:
get_ladder(10)